In [1]:
# resource: http://spark.apache.org/docs/latest/ml-classification-regression.html#linear-regression
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [2]:
from pyspark.ml.regression import LinearRegression

In [3]:
training = spark.read.format('libsvm').load('FileStore/tables/sample_linear_regression_data.txt')

In [4]:
training.show()

+-------------------+--------------------+
 label| features|
+-------------------+--------------------+
 -9.490009878824548|(10,[0,1,2,3,4,5,...|
 0.2577820163584905|(10,[0,1,2,3,4,5,...|
 -4.438869807456516|(10,[0,1,2,3,4,5,...|
-19.782762789614537|(10,[0,1,2,3,4,5,...|
 -7.966593841555266|(10,[0,1,2,3,4,5,...|
 -7.896274316726144|(10,[0,1,2,3,4,5,...|
 -8.464803554195287|(10,[0,1,2,3,4,5,...|
 2.1214592666251364|(10,[0,1,2,3,4,5,...|
 1.0720117616524107|(10,[0,1,2,3,4,5,...|
-13.772441561702871|(10,[0,1,2,3,4,5,...|
 -5.082010756207233|(10,[0,1,2,3,4,5,...|
 7.887786536531237|(10,[0,1,2,3,4,5,...|
 14.323146365332388|(10,[0,1,2,3,4,5,...|
-20.057482615789212|(10,[0,1,2,3,4,5,...|
-0.8995693247765151|(10,[0,1,2,3,4,5,...|
 -19.16829262296376|(10,[0,1,2,3,4,5,...|
 5.601801561245534|(10,[0,1,2,3,4,5,...|
-3.2256352187273354|(10,[0,1,2,3,4,5,...|
 1.5299675726687754|(10,[0,1,2,3,4,5,...|
 -0.250102447941961|(10,[0,1,2,3,4,5,...|
+-------------------+--------------------+
only showing top 20 rows

In [5]:
lr = LinearRegression(featuresCol = 'features', labelCol = 'label', predictionCol = 'prediction')

In [6]:
lrModel = lr.fit(training)

In [7]:
lrModel.coefficients

Out[7]: DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [8]:
lrModel.intercept

Out[8]: 0.14228558260358093

In [9]:
training_summary = lrModel.summary

In [10]:
# evaluate metrics
training_summary.r2

Out[10]: 0.027839179518600154

In [11]:
training_summary.rootMeanSquaredError

Out[11]: 10.16309157133015

In [12]:
# train-test split
all_data = spark.read.format('libsvm').load('FileStore/tables/sample_linear_regression_data.txt')

In [13]:
# randomly split the train/test dataset
split_object = all_data.randomSplit([0.7,0.3])
split_object

Out[14]: [DataFrame[label: double, features: vector],
 DataFrame[label: double, features: vector]]

In [14]:
train_data, test_data = all_data.randomSplit([0.7,0.3])

In [15]:
train_data.describe().show()

+-------+-------------------+
summary| label|
+-------+-------------------+
 count| 338|
 mean| 0.9304401913604055|
 stddev| 10.5205174446124|
 min|-28.046018037776633|
 max| 27.78383192005107|
+-------+-------------------+

In [16]:
test_data.describe().show()

+-------+-------------------+
summary| label|
+-------+-------------------+
 count| 163|
 mean|-1.1398005200007744|
 stddev| 9.768448788738791|
 min|-28.571478869743427|
 max| 22.31738046492344|
+-------+-------------------+

In [17]:
# fit on training data
correct_model = lr.fit(train_data)

In [18]:
# evaluate on testing data
test_results = correct_model.evaluate(test_data)

In [19]:
test_results.residuals.show()

+-------------------+
 residuals|
+-------------------+
 -30.41323386823524|
-25.656751057976127|
-25.825921206499515|
-20.659595716331925|
-21.923714909444104|
-20.629705608021784|
-20.245934368172186|
-19.610236654276534|
-18.398512129375664|
-14.108172968399503|
 -17.74229099546971|
-17.413171634838793|
-16.220262351033163|
-20.621020729402744|
-17.237126677300548|
 -16.21017699918199|
 -16.26409927200594|
-14.840662455882796|
-19.496585291466396|
 -12.92169048449691|
+-------------------+
only showing top 20 rows

In [20]:
test_results.rootMeanSquaredError

Out[22]: 10.015778917515414

In [21]:
# deploy a model
unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
 features|
+--------------------+
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows

In [22]:
predictions = correct_model.transform(unlabeled_data)

In [23]:
predictions.show()

+--------------------+--------------------+
 features| prediction|
+--------------------+--------------------+
(10,[0,1,2,3,4,5,...| 1.8417549984918118|
(10,[0,1,2,3,4,5,...| -1.0794561246255965|
(10,[0,1,2,3,4,5,...| 2.8760952703034417|
(10,[0,1,2,3,4,5,...| -2.1778647005874183|
(10,[0,1,2,3,4,5,...| 0.491327145278298|
(10,[0,1,2,3,4,5,...| 0.74514483374836|
(10,[0,1,2,3,4,5,...| 0.5786157528004656|
(10,[0,1,2,3,4,5,...| 2.181562083337029|
(10,[0,1,2,3,4,5,...| 1.071791396699716|
(10,[0,1,2,3,4,5,...| -2.9183192958100457|
(10,[0,1,2,3,4,5,...| 1.590941644192597|
(10,[0,1,2,3,4,5,...| 1.3275125938173045|
(10,[0,1,2,3,4,5,...| 0.26874978523858994|
(10,[0,1,2,3,4,5,...| 4.840335696779443|
(10,[0,1,2,3,4,5,...| 1.505038405061303|
(10,[0,1,2,3,4,5,...| 0.7727922057507718|
(10,[0,1,2,3,4,5,...| 1.2076162974635059|
(10,[0,1,2,3,4,5,...| 0.9735745607240291|
(10,[0,1,2,3,4,5,...| 5.724143729763524|
(10,[0,1,2,3,4,5,...|-0.11823757960770509|
+--------------------+--------------------+
only showing top 20 rows

In [24]:
# a more realistic model
# explore data from an e-commerce company's website and mobile app
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [25]:
from pyspark.ml.regression import LinearRegression

In [26]:
data = spark.read.csv('FileStore/tables/Ecommerce_Customers.csv', inferSchema = True, header = True)
data.printSchema()

root
-- Email: string (nullable = true)
-- Address: string (nullable = true)
-- Avatar: string (nullable = true)
-- Avg Session Length: double (nullable = true)
-- Time on App: double (nullable = true)
-- Time on Website: double (nullable = true)
-- Length of Membership: double (nullable = true)
-- Yearly Amount Spent: double (nullable = true)

In [27]:
for item in data.head(1)[0]:
  print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005

In [28]:
# setup the dataframe for MLlib
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [29]:
# convert all the numerical values into one list
data.columns

Out[41]: ['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [30]:
# take all the x's to generate one vector
assembler = VectorAssembler(inputCols = ['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership'],
                         outputCol = 'features')

In [31]:
output = assembler.transform(data)

In [32]:
output.printSchema()

root
-- Email: string (nullable = true)
-- Address: string (nullable = true)
-- Avatar: string (nullable = true)
-- Avg Session Length: double (nullable = true)
-- Time on App: double (nullable = true)
-- Time on Website: double (nullable = true)
-- Length of Membership: double (nullable = true)
-- Yearly Amount Spent: double (nullable = true)
-- features: vector (nullable = true)

In [33]:
output.head(1)
# a new feature is created

Out[46]: [Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [34]:
final_data = output.select('features', 'Yearly Amount Spent')

In [35]:
final_data.show()

+--------------------+-------------------+
 features|Yearly Amount Spent|
+--------------------+-------------------+
[34.4972677251122...| 587.9510539684005|
[31.9262720263601...| 392.2049334443264|
[33.0009147556426...| 487.54750486747207|
[34.3055566297555...| 581.8523440352177|
[33.3306725236463...| 599.4060920457634|
[33.8710378793419...| 637.102447915074|
[32.0215955013870...| 521.5721747578274|
[32.7391429383803...| 549.9041461052942|
[33.9877728956856...| 570.2004089636196|
[31.9365486184489...| 427.1993848953282|
[33.9925727749537...| 492.6060127179966|
[33.8793608248049...| 522.3374046069357|
[29.5324289670579...| 408.6403510726275|
[33.1903340437226...| 573.4158673313865|
[32.3879758531538...| 470.4527333009554|
[30.7377203726281...| 461.7807421962299|
[32.1253868972878...| 457.84769594494855|
[32.3388993230671...| 407.70454754954415|
[32.1878120459321...| 452.3156754800354|
[32.6178560628234...| 605.061038804892|
+--------------------+-------------------+
only showing top 20 rows

In [36]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [37]:
train_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 345|
 mean| 502.06726705949785|
 stddev| 76.2604537130915|
 min| 275.9184206503857|
 max| 765.5184619388373|
+-------+-------------------+

In [38]:
test_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 155|
 mean| 493.18588383076514|
 stddev| 85.66411326970575|
 min| 256.67058229005585|
 max| 712.3963268096637|
+-------+-------------------+

In [39]:
lr = LinearRegression(labelCol = 'Yearly Amount Spent')

In [40]:
lr_model = lr.fit(train_data)

In [41]:
test_results = lr_model.evaluate(test_data)

In [42]:
test_results.residuals.show()

+-------------------+
 residuals|
+-------------------+
 11.59516008810374|
 10.771136214099556|
-17.683511496800634|
 -4.185469755564952|
 5.002544958713202|
-12.655186035259419|
 3.889481039631505|
 0.8493904626679978|
 -8.837114544342683|
 3.7427144480680568|
-1.4807182739895097|
 6.685057313283949|
 -6.055297175127066|
 -6.357477359630536|
 -4.263993307943167|
 -4.470699570024635|
 7.400904089598441|
-2.4839706645254864|
-17.205330980577685|
 -4.727531125628161|
+-------------------+
only showing top 20 rows

In [43]:
test_results.r2

Out[56]: 0.9861732116071916

In [44]:
test_results.rootMeanSquaredError

Out[57]: 10.04047116590531

In [45]:
final_data.describe().show()
# compare the RMSE with the original y's distribution
# will find that the RMSE indicates the model's error is pretty small
# note that RMSE has the same unit as the original data, therefore we can compare

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 500|
 mean| 499.3140382585909|
 stddev| 79.3147815497068|
 min| 256.67058229005585|
 max| 765.5184619388373|
+-------+-------------------+

In [46]:
# deploy the model (i.e. predict with unlabeled data)
unlabeled_data = test_data.select('features')
predictions = lr_model.transform(unlabeled_data)
predictions.show()

+--------------------+------------------+
 features| prediction|
+--------------------+------------------+
[29.5324289670579...|397.04519098452374|
[30.7377203726281...| 451.0096059821303|
[30.8162006488763...|283.76985244526963|
[30.8364326747734...|471.68737018255456|
[31.0472221394875...| 387.4948542303082|
[31.0662181616375...| 461.5884792429338|
[31.3662121671876...| 426.6994015168534|
[31.3895854806643...| 409.2202205973149|
[31.5261978982398...| 417.9316407366805|
[31.5316044825729...| 432.7728912812945|
[31.5761319713222...| 542.7073022633178|
[31.6548096756927...|468.57836641426456|
[31.7207699002873...| 544.83023065315|
[31.7242025238451...|509.74536464759103|
[31.7656188210424...| 500.8180749435503|
[31.8124825597242...|397.28104455382186|
[31.8209982016720...| 417.2743769236149|
[31.8530748017465...|461.76909412687746|
[31.9048571310136...| 491.1551884033938|
[31.9453957483445...| 661.7474550632801|
+--------------------+------------------+
only showing top 20 rows